# Silicon Valley Rent Segmentation

## Introduction

The Silicon Valley is one of the world's hub for technology and innovation. From long time companies like Intel and Hewlett-Packard, social media giants like Google, Facebook and Twitter to a plethora of startups - the Silicon Valley attracts high-skilled workers from all over the world.

When there is an influx of workforce into a region, the cost of living begins to increase. The Silicon Valley is currently going through a 'housing crisis' - with housing prices skyrocketing, affordability is a major issue. For someone moving to the Silicon Valley for a new job, buying a home might far from reach. The other option is finding a place to rent.

## Business Problem

In this study, we will be looking at how median rent for a two-bedroom apartment varies within each zip code of the Santa Clara County. For those who are new and moving to the area, it is best to know where the house rents are relatively low but also closer to social venues like restaurants, theatres, malls, parks, etc. This analysis plans to address this issue.

## Data Requirement and Collection

For this study I chose to divide Santa Clara county by zip code (since neighborhood were too small in size and cities with too big in size). Once I have the relevant zip codes for the respective regions in the county I need to merge the data with the median rent for a two-bedroom apartment. Further using Geocoder, find the coordinates for each zip code region and form this into a data set. Finally I will need data about the different venues in each region to help cluster similar regions. I will use Foursquare API to gather this information.

Once the data has been colected and processed, one should be able to view of a map with similar venue categories overlayed over a choropleth showing the median rent of that region.

**For example**, you can look for regions with Mexican restaurants and gyms in the neighborhood and comapre what the median rent in those regions are (this is a hypothetical example prior to running the code).

### The following data must be collected:
#### 1. A list of zip codes within the Santa Clara County
*I was able to scrap data from Santa Clara County Public Health website: https://www.sccgov.org/sites/phd/hi/hd/Pages/city-profiles.aspx. This provided a list of cities and neighborhoods within Santa Clara County along with zipcodes*
#### 2. Median rent for Santa Clara County by Zip code
*This data was scrapped from a real estate search website: https://www.zillow.com/santa-clara-county-ca/home-values/. This website provides the most upto date data on housing prices and rents. I was able to scrap the data for 2019 median rent for each zip code in the county.*
#### 3. Geo Locator for geographical coordinates of the county
*Importing Geocoder library to get the coordinates for Santa Clara County*
#### 4. List of venues in each neighborhood
*A list of venues in each neighborhood will be pulled from Foursquare API*
#### 5. Map generation
*Using Folium to generate maps in Python. In order to plot a choropleth a json file of Santa Clara county was obtained from https://geo.nyu.edu/catalog/stanford-qb712rn3742*


## Importing Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
# The code was removed by Watson Studio for sharing.

## Importing Dataset as pandas

In [16]:
body = client_8ba116acdcca4616a4acef841eb821f1.get_object(Bucket='capstoneproject-donotdelete-pr-gec6h5x5axg3n2',Key='Santa Clara County Rents.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data = pd.read_csv(body)
df_data.head()

,Zip Code,Type,Single family Home,Rent
0,94022.0,Los Altos,"$2,140,800","$4,397"
1,94024.0,Los Altos,"$2,632,500","$3,710"
2,94028.0,Los Altos,"$1,976,800","$5,241"
3,94040.0,Mountain View,"$1,084,500","$3,761"
4,94041.0,Mountain View,"$1,615,500","$3,665"


## Pre-processing the data

In [17]:
df_data.dropna(axis=0, inplace=True)
df_data['Zip Code'] = df_data['Zip Code'].astype(int)
df_data.rename(columns={'Type':'City'}, inplace=True)
df_data.drop(columns='Single family Home', axis=1, inplace=True)
df_data['Rent']=df_data['Rent'].str.replace('$','')
df_data['Rent']=df_data['Rent'].str.replace(',','')
df_data['Rent']=df_data['Rent'].astype(int)
df_data.head()

,Zip Code,City,Rent
0,94022,Los Altos,4397
1,94024,Los Altos,3710
2,94028,Los Altos,5241
3,94040,Mountain View,3761
4,94041,Mountain View,3665


## Get Geographical Coordinates

In [27]:
!pip install geocoder
import geocoder as geocoder
def getLatLong(row):

    # initialize your variable to None
    ll = None
    zipcode = '{}, CA'.format(row)
    # loop until you get the coordinates
    try:
        while(ll is None):
            g = geocoder.arcgis(zipcode)
            ll = g.latlng
            
    except IndexError:
        latitude = 0.0
        longitude = 0.0
        print('BACKUP')
        return [latitude,longitude]

    latitude = ll[0]
    longitude = ll[1]
    print(latitude, longitude)
    return [latitude, longitude]

In [28]:
coords_list = df_data['Zip Code'].apply(getLatLong).tolist()

37.365997400000026 -122.15216969999994
37.35705410000003 -122.09433159999998
37.37551060000004 -122.2220754
37.37722000000008 -122.08765499999998
37.38798450000007 -122.07389959999995
37.42026500000003 -122.07154499999996
37.38884400000006 -122.01760199999995
37.36964500000005 -122.01482499999997
37.35219500000005 -122.03484999999995
37.41770500000007 -122.02675999999997
37.442270000000065 -122.15507999999994
37.44054220000004 -122.11869479999996
37.42360000000008 -122.17142499999994
37.413275000000056 -122.13088999999997
37.610697800000025 -121.67998579999994
37.27898270000003 -121.95395579999996
37.295937200000026 -122.08812719999997
37.039315000000045 -121.54463999999996
36.80712200000005 -121.36270299999995
37.23017150000004 -122.0014473
37.22758570000008 -121.94430699999998
37.172609900000054 -121.94263419999999
37.43960500000003 -121.88158999999996
37.15815310000005 -121.64076269999998
37.09702944700007 -121.57603748799994
37.35652500000003 -121.95046499999995
37.350932800000066 